# DNN

#### Preprocesamiento de las imagenes

##### Convertimos el formato de la imagen a jpg para trabajar con ellas


In [13]:
from cairosvg import svg2png
from PIL import Image
import numpy as np
import io
import os

def convert_svg_to_png(folder_path, output_size=(128, 128)):
    images = {}
    svg_files = [f for f in os.listdir(folder_path) if f.endswith('.svg')]
    
    for svg_file in svg_files:
        # Convert SVG to PNG
        svg_input_path = os.path.join(folder_path, svg_file)
        png_output = svg2png(url=svg_input_path)
        
        # Load PNG into PIL Image and resize
        image = Image.open(io.BytesIO(png_output)).convert('RGB')
        image = image.resize(output_size)
        
        # Store in dict with the filename as key
        images[svg_file.split('.')[0]] = np.array(image)
    
    return images
extract_folder='../../logos'
# Convert all SVG images and save as numpy arrays
converted_images = convert_svg_to_png(extract_folder)
list(converted_images.keys()), list(converted_images.values())[0].shape  # Display first 5 keys and shape of one image


(['ac-milan',
  'ajax',
  'atalanta',
  'atletico-madrid',
  'barcelona',
  'bayern-munchen',
  'benfica',
  'besiktas',
  'chelsea',
  'club-brugge',
  'dynamo-kiyv',
  'fc-porto',
  'internazionale-milano',
  'juventus',
  'liverpool',
  'losc-lille',
  'malmo-ff',
  'manchester-city',
  'manchester-united',
  'paris-saint-germain-f-c',
  'rb-leipzig',
  'real-madrid',
  'redbull-salzburg',
  'sevilla',
  'shakhtar-donetsk',
  'sheriff-tiraspol',
  'sporting-cp',
  'villareal',
  'wolfsburg',
  'youngboys',
  'zenit'],
 (128, 128, 3))

: 